In [1]:
import os
import sys
import numpy as np
import pandas as pd

sys.path.append('../')
sys.path.append('../src/')
sys.path.append('../spell/')

import Reader
import ParamsExtractor3
import DataPreprocessor
import EncodeCommand
import DeepLearningAnomalyDetection2

2024-08-07 08:32:25.531238: I tensorflow/core/util/port.cc:113] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-08-07 08:32:25.581284: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F AVX512_VNNI FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-08-07 08:32:26.402646: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


In [2]:
file_paths = ['../data/laurel_short_' + str(i) for i in range(8)]
existing_file_paths = [path for path in file_paths if os.path.exists(path)]
print(existing_file_paths)

anomalous_fp = '../data/laurel_short_anomalous'

['../data/laurel_short_0', '../data/laurel_short_1', '../data/laurel_short_2', '../data/laurel_short_3', '../data/laurel_short_4', '../data/laurel_short_5', '../data/laurel_short_6', '../data/laurel_short_7']


In [3]:
df_list = []

for f in existing_file_paths:
    df = pd.read_csv(f)
    df_list.append(df)

anomalous_df = pd.read_csv(anomalous_fp)

In [4]:
df_list[0].head(3)

,severity_scores,timedelta,suid,cap_fp,exit,n_dang,n_dang_no_cron,fp_length,col_0,col_1,...,col_12,col_13,col_14,col_15,col_16,col_17,col_18,col_19,enc_comm,enc_parent_comm
0,1.0,0.0,126858,0,0,2,0,92,0,0,...,0,0,0,0,1,0,0,0,1,39
1,1.0,0.0,0,0,0,0,0,38,0,0,...,0,0,0,0,1,0,0,0,2,82
2,1.0,0.0,0,0,0,0,0,28,0,0,...,0,0,0,0,1,0,0,0,3,2


In [5]:
anomalous_df.head(3)

,timedelta,suid,cap_fp,exit,log key,n_dang,n_dang_no_cron,fp_length,enc_comm,enc_parent_comm
0,NaN,998,0,0,-1,0,0,0,1,21
1,0.044,998,0,0,-1,0,0,0,1,21
2,-0.175,42,0,0,-1,0,0,0,2,21


In [6]:
def preprocess_suid_col(df):
    suid_list = df['suid']
    suid_new = [0 if suid_list[i] == 0 else 1 for i in range(len(suid_list))]
    df['suid'] = suid_new

In [7]:
def preprocess_capfp_col(df):
    cap_list = df['cap_fp']
    cap_new = [0 if cap_list[i] == 0 else 1 for i in range(len(cap_list))]
    df['cap_fp'] = cap_new

In [8]:
import pandas as pd

def preprocess_integers(df, col):
    new_col = 'binary_representation_' + col
    df[new_col] = df[col].apply(lambda x: format(int(x), 'b') if not pd.isna(x) else 0)

    # Determine the maximum length of binary strings
    # max_length = df[new_col].astype(str).str.len().max()
    max_length = 12

    # Pad binary strings with zeros to match the maximum length
    df[new_col] = df[new_col].apply(lambda x: x.zfill(max_length) if x is not None else 0)

    # Split binary strings into lists of characters and create a DataFrame
    binary_df = df[new_col].str.split('', expand=True).drop(columns=[0])  # Remove empty first column
    binary_df.columns = [f'{new_col}_{i}' for i in range(binary_df.shape[1])]
    binary_df = binary_df.fillna(0)  # Replace missing values with 0
    binary_df = binary_df.map(lambda x: 0 if not isinstance(x, int) else x)
    binary_df = binary_df.astype(int)

    #print(binary_df.head())
    df = df.drop(new_col, axis=1)

    # Concatenate with original DataFrame
    df = pd.concat([df, binary_df], axis=1)
    #print(df.columns)
    
    return df

In [9]:
def preprocess_timedelta(df):
    l = df['timedelta']
    l_new = [abs(x) if not pd.isna(x) else -1 for x in l]
    df['timedelta'] = l_new

In [10]:
for i, df in enumerate(df_list):
    preprocess_suid_col(df)
    preprocess_capfp_col(df)
    preprocess_timedelta(df)
    df_list[i] = preprocess_integers(df, 'enc_comm')
    df_list[i] = preprocess_integers(df_list[i], 'enc_parent_comm')
    df_list[i] = preprocess_integers(df_list[i], 'fp_length')
    df_list[i] = df_list[i].drop(['enc_comm', 'enc_parent_comm', 'fp_length'], axis=1)

In [11]:
df_list[0].columns

Index(['severity_scores', 'timedelta', 'suid', 'cap_fp', 'exit', 'n_dang',
       'n_dang_no_cron', 'col_0', 'col_1', 'col_2', 'col_3', 'col_4', 'col_5',
       'col_6', 'col_7', 'col_8', 'col_9', 'col_10', 'col_11', 'col_12',
       'col_13', 'col_14', 'col_15', 'col_16', 'col_17', 'col_18', 'col_19',
       'binary_representation_enc_comm_0', 'binary_representation_enc_comm_1',
       'binary_representation_enc_comm_2', 'binary_representation_enc_comm_3',
       'binary_representation_enc_comm_4', 'binary_representation_enc_comm_5',
       'binary_representation_enc_comm_6', 'binary_representation_enc_comm_7',
       'binary_representation_enc_comm_8', 'binary_representation_enc_comm_9',
       'binary_representation_enc_comm_10',
       'binary_representation_enc_comm_11',
       'binary_representation_enc_comm_12',
       'binary_representation_enc_parent_comm_0',
       'binary_representation_enc_parent_comm_1',
       'binary_representation_enc_parent_comm_2',
       'binary_repr

In [12]:
preprocess_suid_col(anomalous_df)
preprocess_timedelta(anomalous_df)
preprocess_capfp_col(anomalous_df)
anomalous_df = preprocess_integers(anomalous_df, 'enc_comm')
anomalous_df = preprocess_integers(anomalous_df, 'enc_parent_comm')
anomalous_df = preprocess_integers(anomalous_df, 'fp_length')
anomalous_df = anomalous_df.drop(['enc_comm', 'enc_parent_comm', 'fp_length'], axis=1)

In [13]:
anomalous_df.head()

,timedelta,suid,cap_fp,exit,log key,n_dang,n_dang_no_cron,binary_representation_enc_comm_0,binary_representation_enc_comm_1,binary_representation_enc_comm_2,...,binary_representation_fp_length_3,binary_representation_fp_length_4,binary_representation_fp_length_5,binary_representation_fp_length_6,binary_representation_fp_length_7,binary_representation_fp_length_8,binary_representation_fp_length_9,binary_representation_fp_length_10,binary_representation_fp_length_11,binary_representation_fp_length_12
0,-1.000,1,0,0,-1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0.044,1,0,0,-1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0.175,1,0,0,-1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0.129,1,0,0,-1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0.124,1,0,0,-1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [14]:
anomalous_df.columns

Index(['timedelta', 'suid', 'cap_fp', 'exit', 'log key', 'n_dang',
       'n_dang_no_cron', 'binary_representation_enc_comm_0',
       'binary_representation_enc_comm_1', 'binary_representation_enc_comm_2',
       'binary_representation_enc_comm_3', 'binary_representation_enc_comm_4',
       'binary_representation_enc_comm_5', 'binary_representation_enc_comm_6',
       'binary_representation_enc_comm_7', 'binary_representation_enc_comm_8',
       'binary_representation_enc_comm_9', 'binary_representation_enc_comm_10',
       'binary_representation_enc_comm_11',
       'binary_representation_enc_comm_12',
       'binary_representation_enc_parent_comm_0',
       'binary_representation_enc_parent_comm_1',
       'binary_representation_enc_parent_comm_2',
       'binary_representation_enc_parent_comm_3',
       'binary_representation_enc_parent_comm_4',
       'binary_representation_enc_parent_comm_5',
       'binary_representation_enc_parent_comm_6',
       'binary_representation_enc_pare

In [15]:
columns_for_laurel = anomalous_df.columns
anomalous_dataset_encoded = anomalous_df

In [16]:
for df in df_list:
    print(df.shape)

(1000, 66)
(1000, 66)
(1000, 66)
(1000, 66)
(1000, 66)
(1000, 66)
(1000, 66)
(1000, 66)


In [17]:
normal_dataset = df_list[0]

for i in range(1,len(df_list)):
    normal_dataset = np.vstack([normal_dataset, df_list[i]])
    
print(normal_dataset)
normal_df_encoded = pd.DataFrame(normal_dataset, columns=df_list[0].columns)
normal_df_encoded.head()

[[1. 0. 1. ... 0. 0. 0.]
 [1. 0. 0. ... 0. 0. 0.]
 [1. 0. 0. ... 0. 0. 0.]
 ...
 [1. 0. 0. ... 0. 0. 0.]
 [1. 0. 0. ... 0. 0. 0.]
 [1. 0. 0. ... 0. 0. 0.]]


,severity_scores,timedelta,suid,cap_fp,exit,n_dang,n_dang_no_cron,col_0,col_1,col_2,...,binary_representation_fp_length_3,binary_representation_fp_length_4,binary_representation_fp_length_5,binary_representation_fp_length_6,binary_representation_fp_length_7,binary_representation_fp_length_8,binary_representation_fp_length_9,binary_representation_fp_length_10,binary_representation_fp_length_11,binary_representation_fp_length_12
0,1.0,0.0,1.0,0.0,0.0,2.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [18]:
normal_df_encoded.columns

Index(['severity_scores', 'timedelta', 'suid', 'cap_fp', 'exit', 'n_dang',
       'n_dang_no_cron', 'col_0', 'col_1', 'col_2', 'col_3', 'col_4', 'col_5',
       'col_6', 'col_7', 'col_8', 'col_9', 'col_10', 'col_11', 'col_12',
       'col_13', 'col_14', 'col_15', 'col_16', 'col_17', 'col_18', 'col_19',
       'binary_representation_enc_comm_0', 'binary_representation_enc_comm_1',
       'binary_representation_enc_comm_2', 'binary_representation_enc_comm_3',
       'binary_representation_enc_comm_4', 'binary_representation_enc_comm_5',
       'binary_representation_enc_comm_6', 'binary_representation_enc_comm_7',
       'binary_representation_enc_comm_8', 'binary_representation_enc_comm_9',
       'binary_representation_enc_comm_10',
       'binary_representation_enc_comm_11',
       'binary_representation_enc_comm_12',
       'binary_representation_enc_parent_comm_0',
       'binary_representation_enc_parent_comm_1',
       'binary_representation_enc_parent_comm_2',
       'binary_repr

In [19]:
normal_dataset_df = pd.DataFrame(normal_df_encoded, columns=columns_for_laurel)
normal_dataset_df.fillna(-1, inplace=True)
anomalous_dataset_df = pd.DataFrame(anomalous_dataset_encoded, columns=columns_for_laurel)
#anomalous_dataset_df.fillna(0, inplace=True)

In [20]:
normal_dataset_df.head()

,timedelta,suid,cap_fp,exit,log key,n_dang,n_dang_no_cron,binary_representation_enc_comm_0,binary_representation_enc_comm_1,binary_representation_enc_comm_2,...,binary_representation_fp_length_3,binary_representation_fp_length_4,binary_representation_fp_length_5,binary_representation_fp_length_6,binary_representation_fp_length_7,binary_representation_fp_length_8,binary_representation_fp_length_9,binary_representation_fp_length_10,binary_representation_fp_length_11,binary_representation_fp_length_12
0,0.0,1.0,0.0,0.0,-1.0,2.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0,-1.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,-1.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,-1.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,-1.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [21]:
'''
d_an_det = DeepLearningAnomalyDetection2.DeepLearningAnomalyDetection()
true_anomalies, reconstructed_anomalies, fp_ = d_an_det.train_test_model(normal_dataset_df, anomalous_dataset_df, 'autoencoder', laurel=1, plots=[0,0,1,1])
'''

"\nd_an_det = DeepLearningAnomalyDetection2.DeepLearningAnomalyDetection()\ntrue_anomalies, reconstructed_anomalies, fp_ = d_an_det.train_test_model(normal_dataset_df, anomalous_dataset_df, 'autoencoder', laurel=1, plots=[0,0,1,1])\n"

In [22]:
missing_values_count = normal_dataset_df.isna().sum().sum()
print(missing_values_count)

0


In [23]:
normal_dataset_df.head()

,timedelta,suid,cap_fp,exit,log key,n_dang,n_dang_no_cron,binary_representation_enc_comm_0,binary_representation_enc_comm_1,binary_representation_enc_comm_2,...,binary_representation_fp_length_3,binary_representation_fp_length_4,binary_representation_fp_length_5,binary_representation_fp_length_6,binary_representation_fp_length_7,binary_representation_fp_length_8,binary_representation_fp_length_9,binary_representation_fp_length_10,binary_representation_fp_length_11,binary_representation_fp_length_12
0,0.0,1.0,0.0,0.0,-1.0,2.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0,-1.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,-1.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,-1.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,-1.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [24]:
anomalous_dataset_df.head()

,timedelta,suid,cap_fp,exit,log key,n_dang,n_dang_no_cron,binary_representation_enc_comm_0,binary_representation_enc_comm_1,binary_representation_enc_comm_2,...,binary_representation_fp_length_3,binary_representation_fp_length_4,binary_representation_fp_length_5,binary_representation_fp_length_6,binary_representation_fp_length_7,binary_representation_fp_length_8,binary_representation_fp_length_9,binary_representation_fp_length_10,binary_representation_fp_length_11,binary_representation_fp_length_12
0,-1.000,1,0,0,-1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0.044,1,0,0,-1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0.175,1,0,0,-1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0.129,1,0,0,-1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0.124,1,0,0,-1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [25]:
import TimeDependentDeepLearningAnomalyDetection as td

In [26]:
detector = td.TimeDependentDeepLearningAnomalyDetection()
detector.analyze(normal_dataset_df)

46
(7941, 50, 46) (7941, 10, 46)


Model: "CONV_LSTM_model"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ input_layer (InputLayer)        │ (None, 50, 46)         │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ lstm (LSTM)                     │ (None, 50, 40)         │        13,920 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv1d (Conv1D)                 │ (None, 50, 46)         │        18,446 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ cropping (Cropping1D)           │ (None, 10, 46)         │             0 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 32,366 (126.43 KB)

 Trainable params: 32,366 (126.43 KB)

 Non-trainable params: 0 (0.00 B)

Epoch 1/100
159/159 ━━━━━━━━━━━━━━━━━━━━ 3s 13ms/step - loss: 0.0539 - val_loss: 0.0441
Epoch 2/100
159/159 ━━━━━━━━━━━━━━━━━━━━ 2s 11ms/step - loss: 0.0393 - val_loss: 0.0435
Epoch 3/100
159/159 ━━━━━━━━━━━━━━━━━━━━ 3s 11ms/step - loss: 0.0381 - val_loss: 0.0445
Epoch 4/100
159/159 ━━━━━━━━━━━━━━━━━━━━ 2s 11ms/step - loss: 0.0374 - val_loss: 0.0463
Epoch 5/100
159/159 ━━━━━━━━━━━━━━━━━━━━ 2s 11ms/step - loss: 0.0367 - val_loss: 0.0544
Epoch 6/100
159/159 ━━━━━━━━━━━━━━━━━━━━ 2s 11ms/step - loss: 0.0361 - val_loss: 0.0555
Epoch 7/100
159/159 ━━━━━━━━━━━━━━━━━━━━ 2s 12ms/step - loss: 0.0356 - val_loss: 0.0536
Epoch 8/100
159/159 ━━━━━━━━━━━━━━━━━━━━ 2s 11ms/step - loss: 0.0352 - val_loss: 0.0533
Epoch 9/100
159/159 ━━━━━━━━━━━━━━━━━━━━ 2s 11ms/step - loss: 0.0347 - val_loss: 0.0519
Epoch 10/100
159/159 ━━━━━━━━━━━━━━━━━━━━ 2s 11ms/step - loss: 0.0344 - val_loss: 0.0528
Epoch 11/100
159/159 ━━━━━━━━━━━━━━━━━━━━ 2s 11ms/step - loss: 0.0340 - val_loss: 0.0524
Epoch 12/100
159/159 ━━━━━━━━━

0.03585993

In [27]:
an_X, an_y = detector.create_sequences(anomalous_dataset_df)

(479, 50, 46) (479, 10, 46)


In [28]:
an_y_pred = detector.predict(an_X)
mse = detector.evaluate(an_y, an_y_pred)

15/15 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step
Mean Squared Error: 0.09624782986366572
